# Generador de descuentos

## Objetivos

* Incentivar nuevas compras del cliente en el establecimiento

* Fomentar el consumo de otros productos

* Fomentar el consumo de productos con más margen de beneficio


## Entradas y Salidas

* **Entrada:** Lista de artículos que ha comprado el consumidor
* **Salida:** Lista de cupones descuento que imprimir junto al recibo de compra

In [88]:
from experta import *
import re

## Hechos

Definiremos a continuación los hechos que manejará el sistema.

In [89]:
class Producto(Fact):
    """
    Producto que ha comprado un cliente.

    >>> Producto(nombre="pepsi", tipo="refresco de cola", cantidad=1)

    """
    pass

class Cupon(Fact):
    """
    Cupón a generar para la próxima compra del cliente.

    >>> Cupon(tipo="2x1", producto="pepsi")
    
    """
    pass


In [90]:
class Promo(Fact):
    """
    Promoción vigente en el comercio.

    >>> Promo(tipo="2x1", **depende_de_la_promo)

    """
    pass

class Beneficio(Fact):
    """
    Define los beneficios que obtiene el comercio por cada producto.

    >>> Beneficio(nombre="pepsi", tipo="refresco de cola", ganancias=0.2)

    """
    pass

## Hechos (funcionalidad(es))

In [91]:
class TarjetaP(Fact):
    """
    En esta clase se almacenarán los puntos del cliente para poder ser redimidos

    >>> TarjetaP(documento = "1000134735", nombre = "Íngrid Mejía", puntos_tarjeta=500)

    """
    pass

In [92]:
class Puntos(Fact):
    """
    Algunos productos tendrán asociados puntos que podrán ser redimidos más adelante.

    >>> Puntos(producto = "Camisetas para Hombre - GEF", puntos_redimibles = 300)

    """
    pass

## Objetivo 1
### Incentivar nuevas compras del cliente en el establecimiento
Para esto no hay nada mejor que las típicas promociones **2x1**, **3x2**, etc.

#### Implementación

In [93]:
class OfertasNxM(KnowledgeEngine):
    @DefFacts()
    def carga_promociones_nxm(self):
        """
        Hechos iniciales.
        
        Genera las promociones vigentes
        """
        yield Promo(tipo="2x1", producto="Dodot")
        yield Promo(tipo="2x1", producto="Leche Pascual")
        yield Promo(tipo="3x2", producto="Pilas AAA")
    
    @Rule(Promo(tipo=MATCH.t & P(lambda t: re.match(r"\d+x\d+", t)),
                producto=MATCH.p),
          Producto(nombre=MATCH.p))
    def oferta_nxm(self, t, p):
        """
        Sabemos que el cliente volverá para aprovechar
        la promoción, ya que hoy ha comprado el producto.
        """
        self.declare(Cupon(tipo=t, producto=p))

## Objetivo 1 (funcionalidad(es))

In [94]:
class ProcesoPuntos(KnowledgeEngine):
    @DefFacts()
    def carga_tarjetas(self):
        """"
        Hechos iniciales.
        
        Genera las tarjetas de los clientes que decidieron probar esta funcionalidad
        """
        yield TarjetaP(documento = "1000134735", nombre = "Íngrid Mejía", puntos_tarjeta = 500)
        yield TarjetaP(documento = "1000134736", nombre = "Luis Córdoba", puntos_tarjeta = 1000)
        yield TarjetaP(documento = "1000134737", nombre = "Angela Garzon", puntos_tarjeta = 750)

        
    @DefFacts()
    def carga_productos_asociados(self):
        """
        Hechos iniciales.
        
        Genera y asocia los puntos a los productos asignados
        """
        yield Puntos(producto = "Camisetas para Hombre - GEF", puntos_redimibles = 300)
        yield Puntos(producto = "Camiseta Ms Mujer GEF 720478", puntos_redimibles = 250)
        yield Puntos(producto = "Tennis Nike Air Zoom Pegasus 39", puntos_redimibles = 500)
    
    """    
    Compara los documentos y los puntos en ellos
    """
    @Rule(
        TarjetaP(documento = MATCH.documento, puntos_tarjeta = MATCH.puntos_tarjeta),
        Puntos(producto = MATCH.producto, puntos_redimibles = MATCH.puntos_redimibles)
        
    )
    def asociacion_puntos(self, puntos_tarjeta, puntos_redimibles):
        """  
        Suma los puntos de la tarjeta del cliente y los con los puntos que da cada producto

        """
        self.declare(TarjetaP(puntos_tarjeta = puntos_tarjeta + puntos_redimibles))
        
        

#### Pruebas
Utilizaremos la función `watch` para ver qué está haciendo el motor durante la ejecución.

In [95]:
watch('RULES', 'FACTS')

In [124]:
nxm = OfertasNxM()
nxm.reset()

ptn = ProcesoPuntos()
ptn.reset()

INFO:experta.watchers.FACTS: ==> <f-0>: InitialFact()
INFO:experta.watchers.FACTS: ==> <f-1>: Promo(tipo='2x1', producto='Dodot')
INFO:experta.watchers.FACTS: ==> <f-2>: Promo(tipo='2x1', producto='Leche Pascual')
INFO:experta.watchers.FACTS: ==> <f-3>: Promo(tipo='3x2', producto='Pilas AAA')
INFO:experta.watchers.FACTS: ==> <f-0>: InitialFact()
INFO:experta.watchers.FACTS: ==> <f-1>: Puntos(producto='Camisetas para Hombre - GEF', puntos_redimibles=300)
INFO:experta.watchers.FACTS: ==> <f-2>: Puntos(producto='Camiseta Ms Mujer GEF 720478', puntos_redimibles=250)
INFO:experta.watchers.FACTS: ==> <f-3>: TarjetaP(documento='1000134735', nombre='Íngrid Mejía', puntos_tarjeta=500)
INFO:experta.watchers.FACTS: ==> <f-4>: TarjetaP(documento='1000134736', nombre='Luis Córdoba', puntos_tarjeta=1000)


In [126]:
nxm.declare(Producto(nombre="Dodot",precio=1000))
ptn.declare(Producto(nombre = "Camisetas para Hombre - GEF", precio = 48500))


INFO:experta.watchers.FACTS: ==> <f-5>: Producto(nombre='Camisetas para Hombre - GEF', precio=48500)


Producto(nombre='Camisetas para Hombre - GEF', precio=48500)

In [98]:
nxm.declare(Producto(nombre="Agua Mineral"))

INFO:experta.watchers.FACTS: ==> <f-5>: Producto(nombre='Agua Mineral')


Producto(nombre='Agua Mineral')

In [99]:
nxm.declare(Producto(nombre="Pilas AAA"))

INFO:experta.watchers.FACTS: ==> <f-6>: Producto(nombre='Pilas AAA')


Producto(nombre='Pilas AAA')

In [127]:
nxm.run()
ptn.run()

INFO:experta.watchers.RULES:FIRE 1 oferta_nxm: <f-4>, <f-1>
INFO:experta.watchers.FACTS: ==> <f-5>: Cupon(tipo='2x1', producto='Dodot')
INFO:experta.watchers.RULES:FIRE 1 asociacion_puntos: <f-4>, <f-2>
INFO:experta.watchers.FACTS: ==> <f-6>: TarjetaP(puntos_tarjeta=1250)
INFO:experta.watchers.RULES:FIRE 2 asociacion_puntos: <f-4>, <f-1>
INFO:experta.watchers.FACTS: ==> <f-7>: TarjetaP(puntos_tarjeta=1300)
INFO:experta.watchers.RULES:FIRE 3 asociacion_puntos: <f-2>, <f-3>
INFO:experta.watchers.FACTS: ==> <f-8>: TarjetaP(puntos_tarjeta=750)
INFO:experta.watchers.RULES:FIRE 4 asociacion_puntos: <f-1>, <f-3>
INFO:experta.watchers.FACTS: ==> <f-9>: TarjetaP(puntos_tarjeta=800)


## Objetivo 2
### Fomentar el consumo de otros productos

Para lograr este objetivo generaremos cupones con packs descuento. Ejemplo:

* Si compras una fregona y una mopa a la vez, tienes un 25% de descuento en ambos productos

#### Implementación

In [101]:
class OfertasPACK(KnowledgeEngine):
    @DefFacts()
    def carga_promociones_pack(self):
        """Genera las promociones vigentes"""
        yield Promo(tipo="PACK", producto1="Fregona ACME", producto2="Mopa ACME", descuento="25%")
        yield Promo(tipo="PACK", producto1="Pasta Gallo", producto2="Tomate Frito", descuento="10%")

    @Rule(Promo(tipo="PACK", producto1=MATCH.p1, producto2=MATCH.p2, descuento=MATCH.d),
          OR(
              AND(
                  NOT(Producto(nombre=MATCH.p1)),
                  Producto(nombre=MATCH.p2)
              ),
              AND(
                  Producto(nombre=MATCH.p1),
                  NOT(Producto(nombre=MATCH.p2))
              )
          )
    )
    def pack(self, p1, p2, d):
        """
        El cliente querrá comprar un producto adicional en su próxima visita.
        """
        self.declare(Cupon(tipo="PACK", producto1=p1, producto2=p2, descuento=d))

#### Pruebas

In [102]:
pack = OfertasPACK()

In [103]:
pack.reset()

INFO:experta.watchers.FACTS: ==> <f-0>: InitialFact()
INFO:experta.watchers.FACTS: ==> <f-1>: Promo(tipo='PACK', producto1='Fregona ACME', producto2='Mopa ACME', descuento='25%')
INFO:experta.watchers.FACTS: ==> <f-2>: Promo(tipo='PACK', producto1='Pasta Gallo', producto2='Tomate Frito', descuento='10%')


In [104]:
pack.declare(Producto(nombre="Tomate Frito"))

INFO:experta.watchers.FACTS: ==> <f-3>: Producto(nombre='Tomate Frito')


Producto(nombre='Tomate Frito')

In [105]:
pack.declare(Producto(nombre="Fregona ACME"))

INFO:experta.watchers.FACTS: ==> <f-4>: Producto(nombre='Fregona ACME')


Producto(nombre='Fregona ACME')

In [106]:
pack.run()

INFO:experta.watchers.RULES:FIRE 1 pack: <f-4>, <f-1>
INFO:experta.watchers.FACTS: ==> <f-5>: Cupon(tipo='PACK', producto1='Fregona ACME', producto2='Mopa ACME', descuento='25%')
INFO:experta.watchers.RULES:FIRE 2 pack: <f-3>, <f-2>
INFO:experta.watchers.FACTS: ==> <f-6>: Cupon(tipo='PACK', producto1='Pasta Gallo', producto2='Tomate Frito', descuento='10%')


## Objetivo 3
### Fomentar el consumo de productos con más margen de beneficio

El truco para cumplir este objetivo es conocer qué beneficio se obtiene por cada producto, y si existe un producto del mismo **tipo** con un beneficio mayor, generar un cupón de descuento para ese producto que nos permita seguir ganando más.

#### Implementación

In [107]:
class OfertasDescuento(KnowledgeEngine):
    @DefFacts()
    def carga_beneficios(self):
        """
        Define las beneficios por producto.
        """
        yield Beneficio(nombre="Mahou", tipo="Cerveza", ganancias=0.5)
        yield Beneficio(nombre="Cerveza Hacendado", tipo="Cerveza", ganancias=0.9)

        yield Beneficio(nombre="Pilas AAA Duracell", tipo="Pilas AAA", ganancias=1.5)
        yield Beneficio(nombre="Pilas AAA Hacendado", tipo="Pilas AAA", ganancias=2)
        
    @Rule(Producto(nombre=MATCH.p1),
          Beneficio(nombre=MATCH.p1, tipo=MATCH.t, ganancias=MATCH.g1),
          Beneficio(nombre=MATCH.p2, tipo=MATCH.t, ganancias=MATCH.g2),
          TEST(lambda g1, g2: g2 > g1)
    )
    def descuento_producto_con_mayor_beneficio(self, p2, g1, g2, **_):
        """
        """
        diferencia_ganancia = g2 - g1
        self.declare(Cupon(tipo="DESCUENTO",
                           producto=p2,
                           cantidad=diferencia_ganancia / 2))

#### Pruebas

In [108]:
descuento = OfertasDescuento()

In [109]:
descuento.reset()

INFO:experta.watchers.FACTS: ==> <f-0>: InitialFact()
INFO:experta.watchers.FACTS: ==> <f-1>: Beneficio(nombre='Mahou', tipo='Cerveza', ganancias=0.5)
INFO:experta.watchers.FACTS: ==> <f-2>: Beneficio(nombre='Cerveza Hacendado', tipo='Cerveza', ganancias=0.9)
INFO:experta.watchers.FACTS: ==> <f-3>: Beneficio(nombre='Pilas AAA Duracell', tipo='Pilas AAA', ganancias=1.5)
INFO:experta.watchers.FACTS: ==> <f-4>: Beneficio(nombre='Pilas AAA Hacendado', tipo='Pilas AAA', ganancias=2)


In [110]:
descuento.declare(Producto(nombre="Mahou"))

INFO:experta.watchers.FACTS: ==> <f-5>: Producto(nombre='Mahou')


Producto(nombre='Mahou')

In [111]:
descuento.run()

INFO:experta.watchers.RULES:FIRE 1 descuento_producto_con_mayor_beneficio: <f-5>, <f-2>, <f-1>
INFO:experta.watchers.FACTS: ==> <f-6>: Cupon(tipo='DESCUENTO', producto='Cerveza Hacendado', cantidad=0.2)


**El sistema no debe generar cupón si se ha comprado el producto con mayor beneficio**

In [112]:
descuento.reset()

INFO:experta.watchers.FACTS: ==> <f-0>: InitialFact()
INFO:experta.watchers.FACTS: ==> <f-1>: Beneficio(nombre='Mahou', tipo='Cerveza', ganancias=0.5)
INFO:experta.watchers.FACTS: ==> <f-2>: Beneficio(nombre='Cerveza Hacendado', tipo='Cerveza', ganancias=0.9)
INFO:experta.watchers.FACTS: ==> <f-3>: Beneficio(nombre='Pilas AAA Duracell', tipo='Pilas AAA', ganancias=1.5)
INFO:experta.watchers.FACTS: ==> <f-4>: Beneficio(nombre='Pilas AAA Hacendado', tipo='Pilas AAA', ganancias=2)


In [113]:
descuento.declare(Producto(nombre="Pilas AAA Hacendado"))

INFO:experta.watchers.FACTS: ==> <f-5>: Producto(nombre='Pilas AAA Hacendado')


Producto(nombre='Pilas AAA Hacendado')

In [114]:
descuento.run()

## Juntándolo todo
Gracias a **Python** podemos utilizar herencia múltiple para unir nuestros distintos motores en uno y darle un mejor interfaz de usuario.

In [115]:
class GeneradorCupones(OfertasNxM, OfertasPACK, OfertasDescuento):
    def generar_cupones(self, *nombre_productos):
        # Reiniciamos el motor
        self.reset()

        # Declaramos los productos que ha comprado el cliente
        for nombre in nombre_productos:
            self.declare(Producto(nombre=nombre))

        # Ejecutamos el motor
        self.run()
        
        # Extraemos las promociones generadas
        for fact in self.facts.values():
            if isinstance(fact, Cupon):
                yield fact

In [116]:
ke = GeneradorCupones()

In [117]:
[cupon for cupon in ke.generar_cupones("Pilas AAA", "Mahou", "Tomate Frito")]

INFO:experta.watchers.FACTS: ==> <f-0>: InitialFact()
INFO:experta.watchers.FACTS: ==> <f-1>: Beneficio(nombre='Mahou', tipo='Cerveza', ganancias=0.5)
INFO:experta.watchers.FACTS: ==> <f-2>: Beneficio(nombre='Cerveza Hacendado', tipo='Cerveza', ganancias=0.9)
INFO:experta.watchers.FACTS: ==> <f-3>: Beneficio(nombre='Pilas AAA Duracell', tipo='Pilas AAA', ganancias=1.5)
INFO:experta.watchers.FACTS: ==> <f-4>: Beneficio(nombre='Pilas AAA Hacendado', tipo='Pilas AAA', ganancias=2)
INFO:experta.watchers.FACTS: ==> <f-5>: Promo(tipo='2x1', producto='Dodot')
INFO:experta.watchers.FACTS: ==> <f-6>: Promo(tipo='2x1', producto='Leche Pascual')
INFO:experta.watchers.FACTS: ==> <f-7>: Promo(tipo='3x2', producto='Pilas AAA')
INFO:experta.watchers.FACTS: ==> <f-8>: Promo(tipo='PACK', producto1='Fregona ACME', producto2='Mopa ACME', descuento='25%')
INFO:experta.watchers.FACTS: ==> <f-9>: Promo(tipo='PACK', producto1='Pasta Gallo', producto2='Tomate Frito', descuento='10%')
INFO:experta.watchers.FAC

[Cupon(tipo='PACK', producto1='Pasta Gallo', producto2='Tomate Frito', descuento='10%'),
 Cupon(tipo='DESCUENTO', producto='Cerveza Hacendado', cantidad=0.2),
 Cupon(tipo='3x2', producto='Pilas AAA')]